In [9]:
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.models import Sequential, Graph
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import normalization
import h5py
from test import *
from keras import backend as K
import time
from FaceRec.pretrained_cnn import *                         # Hi Shivam, this is going to take about a second
import FaceRec
from FaceRec.net import *
from FaceRec.get_input import *
from scipy.misc import imsave
import numpy as np


In [10]:
x,y = from_file()                             # This prints some things on the terminal
x_test, y_test = test_file()                  # last statement must be : No. of images 237 no. of classes: 237

In [11]:
model = Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(3, 227, 227),
                        activation='relu', name='conv1_1', border_mode='same'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))


graph = Graph()
graph.add_input(name='input1', input_shape=(3, 227, 227))
graph.add_node(model, name='cnn', input='input1')

layer_dict = dict([(layer.name, layer) for layer in graph.nodes['cnn'].layers])
# print layer_dict

In [12]:
PRETRAINED = "cnn_weights.h5"
print 'Loading weights ...'
start=time.time()
f = h5py.File(PRETRAINED)
for k in range(f.attrs['nb_layers']):
    if k >= len(graph.nodes['cnn'].layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    graph.nodes['cnn'].layers[k].set_weights(weights)
f.close()
print 'Model loaded in ..',time.time()-start

In [14]:
graph.add_node(Flatten(), input='cnn', name='flatten1')
graph.add_node(Dense(output_dim=4096, activation='relu'), name='dense1', input='flatten1')
graph.add_node(Dense(output_dim=4096, activation='relu'), name='dense2', input='dense1')
graph.add_node(Dropout(0.5), name='dropout1', input='dense2')
graph.add_node(Dense(output_dim=21, activation='softmax'), name='denseOut', input='dropout1')
graph.add_output(name='output', input='denseOut')

In [16]:
print 'Normalizing data...'
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255
y = np_utils.to_categorical(y, 21)
y_test = np_utils.to_categorical(y_test, 21)

IndexError: index 7 is out of bounds for axis 1 with size 7

In [2]:
print """
[[0, 0.99900227785110474],
 [0, 0.99795073270797729],
 [1, 0.99972802400588989],
 [1, 0.9991874098777771],
 [1, 0.99921202659606934],
 [1, 0.99847894906997681],
 [1, 0.99983322620391846],
 [2, 0.99920988082885742],
 [2, 0.99973148107528687],
 [2, 0.99968039989471436],
 [2, 0.99883359670639038],
 [3, 0.99966621398925781],
 [3, 0.99927037954330444],
 [3, 0.99806767702102661],
 [3, 0.99966132640838623],
 [3, 0.99991214275360107],
 [4, 0.99833637475967407],
 [4, 0.99916267395019531],
 [4, 0.99997055530548096],
 [4, 0.99994194507598877],
 [4, 0.99993705749511719],
 [5, 0.99643123149871826],
 [5, 0.99439793825149536],
 [6, 0.9987298846244812],
 [6, 0.99610596895217896],
 [7, 0.99743157625198364],
 [7, 0.99584275484085083],
 [7, 0.99741876125335693],
 [7, 0.98106509447097778],
 [7, 0.92837899923324585],
 [7, 0.93025171756744385],
 [8, 0.99992895126342773],
 [8, 0.99997448921203613],
 [8, 0.99168151617050171],
 [9, 0.99582064151763916],
 [9, 0.9944685697555542],
 [9, 0.99209141731262207],
 [9, 0.98970299959182739],
 [9, 0.98826038837432861],
 [10, 0.9999382495880127],
 [10, 0.9999847412109375],
 [10, 0.99991071224212646],
 [10, 0.99974662065505981],
 [10, 0.99978703260421753],
 [10, 0.99958139657974243],
 [10, 0.99966776371002197],
 [11, 1.0],
 [11, 1.0],
 [11, 1.0],
 [11, 1.0],
 [12, 0.99841654300689697],
 [12, 0.99933749437332153],
 [14, 0.99852591753005981],
 [14, 0.99981313943862915],
 [14, 0.99887043237686157],
 [14, 0.99965262413024902],
 [14, 0.98709326982498169],
 [16, 0.99677747488021851],
 [16, 0.99994409084320068],
 [16, 0.99953162670135498],
 [16, 0.99994170665740967],
 [16, 0.99991524219512939],
 [15, 0.99955183267593384],
 [15, 0.99593544006347656],
 [15, 0.99984645843505859],
 [15, 0.99974876642227173],
 [17, 0.99978739023208618],
 [17, 0.99996650218963623],
 [17, 0.99988925457000732],
 [17, 0.99988484382629395],
 [17, 0.99973124265670776],
 [18, 1.0],
 [18, 1.0],
 [18, 1.0],
 [18, 1.0],
 [18, 0.99999988079071045],
 [18, 1.0],
 [18, 1.0],
 [18, 1.0],
 [18, 1.0],
 [19, 0.9875260591506958],
 [20, 0.99774819612503052],
 [20, 0.99134761095046997],
 [20, 0.9968915581703186],
 [20, 0.99881219863891602],
 [20, 0.99925988912582397],
 [17, 0.99966526031494141],
 [17, 0.99986088275909424],
 [18, 1.0],
 [18, 1.0],
 [18, 1.0],
 [9, 0.53301588535308838],
 [9, 0.84725925922393799],
 [9, 0.98564063549041748],
 [4, 0.69879980087280273],
 [4, 0.99983370304107666],
 [4, 0.99927765130996704],
 [1, 0.99940085411071777],
 [1, 0.99973911046981812],
 [1, 0.99969983100891113]]
"""


[[0, 0.99900227785110474],
 [0, 0.99795073270797729],
 [1, 0.99972802400588989],
 [1, 0.9991874098777771],
 [1, 0.99921202659606934],
 [1, 0.99847894906997681],
 [1, 0.99983322620391846],
 [2, 0.99920988082885742],
 [2, 0.99973148107528687],
 [2, 0.99968039989471436],
 [2, 0.99883359670639038],
 [3, 0.99966621398925781],
 [3, 0.99927037954330444],
 [3, 0.99806767702102661],
 [3, 0.99966132640838623],
 [3, 0.99991214275360107],
 [4, 0.99833637475967407],
 [4, 0.99916267395019531],
 [4, 0.99997055530548096],
 [4, 0.99994194507598877],
 [4, 0.99993705749511719],
 [5, 0.99643123149871826],
 [5, 0.99439793825149536],
 [6, 0.9987298846244812],
 [6, 0.99610596895217896],
 [7, 0.99743157625198364],
 [7, 0.99584275484085083],
 [7, 0.99741876125335693],
 [7, 0.98106509447097778],
 [7, 0.92837899923324585],
 [7, 0.93025171756744385],
 [8, 0.99992895126342773],
 [8, 0.99997448921203613],
 [8, 0.99168151617050171],
 [9, 0.99582064151763916],
 [9, 0.9944685697555542],
 [9, 0.99209141731262207],
 [9

In [3]:
import datetime
print datetime.datetime.now()


2016-05-02 00:35:57.522498
